In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os, sys, copy
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Activation, BatchNormalization, Dropout, Flatten 
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing import image as img_proc

In [24]:
from sklearn.metrics import classification_report, accuracy_score

In [3]:
'''
Define a dictionary which allows us to allocate
labels that we want to each folder that we create.
It contains mapping from folder path to label.

This is included for the continuous labels that we 
plan to allocate to certain samples. 

This represents one 'Scenario'

To create a new scenario, just add/remove/change 
the mappings of filepaths and labels.
'''

label_mapping = {
    'data/Archive/BadData1': 0.0,
    'data/Archive/BadData2': 0.0,
    'data/Archive/GoodData': 1.0,
    'data/Archive/TrueData': 1.0,
    'data/Archive1/TrueGoodData': 1.0
}

## Data generators to generate data on the fly TODO: Implement using Tf.data

In [4]:
'''
We now define a custom DataGenerator class to incorporate 
parallel loading of data on the CPU while GPU trains a batch.

This will prevent overloading the RAM of the system. 
This scheme will act similar to the Datagenerator+Dataloader 
interface of PyTorch.
'''

class DataGenerator(Sequence):
    '''
    Data generator class
    '''
    def __init__(self, label_mapping, batch_size=128, shuffle=True,
                 image_size=(256,256), train_ratio=0.7, 
                 image_preprocess=None):
        
        self.batch_size = batch_size
        self.train_ratio = train_ratio
        self.shuffle = shuffle
        self.data_df = self.get_data_df(label_mapping)
        self.image_size = image_size
        self.image_preprocess = image_preprocess
        self.on_epoch_end()
        
    def set_mode(self, mode='train'):
        
        self.data_df = self.data_df[self.data_df['Mode']==mode]
        self.on_epoch_end()
        
    def get_data_df(self, label_mapping):
        '''
        Produces a dataframe of filepath of each image and
        its label. 
        '''
        
        data_dicts = []
        
        for folder_path in label_mapping:
            
            label = label_mapping[folder_path]
            
            for image_name in os.listdir(folder_path):
                record = {}
                image_path = os.path.join(folder_path, image_name)
                
                if not image_path.endswith('.png'):
                    continue 
                record['ImagePath'] = image_path
                record['Label'] = label
                
                '''
                randomly assign train/val label to each image 
                sampled from uniform distribution over images 
                in each folder and the train ratio
                '''
                if np.random.uniform() > self.train_ratio:
                    record['Mode'] = 'val'
                else:
                    record['Mode'] = 'train'
                
                data_dicts.append(record)
        
        return pd.DataFrame(data_dicts)
    
    def __len__(self):
        '''
        Length of dataset - no. of batches
        '''
        return int(np.ceil(len(self.data_df)/self.batch_size))
        
    def __getitem__(self, index):
        '''
        Get one batch of data 
        '''
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        X, y = self.__data_generation(indexes)
        
        return X, y
        
    def on_epoch_end(self):       
        'Updates indexes after each epoch'
        
        self.indexes = np.arange(len(self.data_df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, indexes):
        '''
        Depending on the indexes, reads images and preprocesses 
        them by the callable image_preprocess and processes the labels
        '''
        batch_df = self.data_df.iloc[indexes]
        image_list = []
        label_list = []
        
        for i in range(len(batch_df)):
            
            try:
            
                image_path = batch_df['ImagePath'].iloc[i]
                label = batch_df['Label'].iloc[i]

                image = img_proc.load_img(image_path, color_mode='grayscale',
                                          target_size=self.image_size)
                img = img_proc.img_to_array(image)
                img = self.image_preprocess(img)
                image_list.append(img)
            
            except:
                print("error in path: {}".format(image_path))
            
            label_ = [1-label, label]
            label_list.append(label_)
        
        return np.moveaxis(np.array(image_list), 1, 3), np.array(label_list)
        

In [5]:
def preprocess_image(image):
    
    image = np.array(image)/255.0
    
    return image

In [6]:
# Generator object to read all files
parent_data_gen = DataGenerator(label_mapping, image_preprocess=preprocess_image)

# Save the data config to csv for further experiments
# Commenting the saving after once saved
# data_df = parent_data_gen.data_df
# data_df.to_csv('data_config.csv', index=None)


In [7]:
# Load the data config using the csv

parent_data_gen.data_df = pd.read_csv('data_config.csv')
parent_data_gen.on_epoch_end()

# Set the train and validation as two copies of the above
train_generator = parent_data_gen
val_generator = copy.deepcopy(parent_data_gen)

# filter the generators data according to partition
train_generator.set_mode('train')
val_generator.set_mode('val')

In [8]:
train_generator.data_df

,ImagePath,Label,Mode
0,data/Archive/BadData1/238.png,0.0,train
1,data/Archive/BadData1/508.png,0.0,train
2,data/Archive/BadData1/517.png,0.0,train
3,data/Archive/BadData1/697.png,0.0,train
4,data/Archive/BadData1/414.png,0.0,train
...,...,...,...
4356,data/Archive1/TrueGoodData/24.png,1.0,train
4357,data/Archive1/TrueGoodData/106.png,1.0,train
4358,data/Archive1/TrueGoodData/961.png,1.0,train
4364,data/Archive1/TrueGoodData/237.png,1.0,train


In [9]:
val_generator.data_df

,ImagePath,Label,Mode
7,data/Archive/BadData1/134.png,0.0,val
8,data/Archive/BadData1/63.png,0.0,val
17,data/Archive/BadData1/675.png,0.0,val
23,data/Archive/BadData1/668.png,0.0,val
25,data/Archive/BadData1/544.png,0.0,val
...,...,...,...
4360,data/Archive1/TrueGoodData/782.png,1.0,val
4361,data/Archive1/TrueGoodData/208.png,1.0,val
4362,data/Archive1/TrueGoodData/359.png,1.0,val
4363,data/Archive1/TrueGoodData/55.png,1.0,val


## Testing the generators

In [10]:
val_generator.__len__()

11

In [11]:
train_generator.__len__()

24

In [12]:
X, y = train_generator.__getitem__(3)

In [13]:
X.shape

(128, 256, 256, 1)

In [14]:
y.shape

(128, 2)

In [15]:
y[0]

array([1., 0.])

In [16]:
input_shape = X.shape[1:]
input_shape

(256, 256, 1)

## CNN iteration zero

In [17]:
model = Sequential()

model.add(Conv2D(8, (3,3), use_bias=False, padding='same',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Conv2D(16, (3,3), use_bias=False, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Conv2D(16, (3,3), use_bias=False, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Conv2D(32, (3,3), use_bias=False, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Conv2D(32, (3,3), use_bias=False, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Conv2D(16, (3,3), use_bias=False, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Conv2D(8, (3,3), use_bias=False, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(2, (3,3), use_bias=False, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Global average pooling
model.add(AveragePooling2D(pool_size=(4,4), strides=(4,4), padding='same'))
model.add(Flatten())
model.add(Activation('softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 8, 256, 1)         18432     
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 256, 1)         4         
_________________________________________________________________
activation (Activation)      (None, 8, 256, 1)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 128, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 128, 1)        1152      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 128, 1)        4         
_________________________________________________________________
activation_1 (Activation)    (None, 16, 128, 1)        0

In [19]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(train_generator, 
                    epochs=100, 
                    validation_data=val_generator,
                    use_multiprocessing=True,
                    workers=os.cpu_count())

Epoch 1/100
24/24 [==============================] - 4s 157ms/step - loss: 0.6755 - accuracy: 0.5906 - val_loss: 0.6924 - val_accuracy: 0.5771
Epoch 2/100
24/24 [==============================] - 3s 137ms/step - loss: 0.6282 - accuracy: 0.6298 - val_loss: 0.6923 - val_accuracy: 0.5771
Epoch 3/100
24/24 [==============================] - 4s 157ms/step - loss: 0.5764 - accuracy: 0.7077 - val_loss: 0.6923 - val_accuracy: 0.5778
Epoch 4/100
24/24 [==============================] - 3s 142ms/step - loss: 0.5183 - accuracy: 0.7644 - val_loss: 0.6924 - val_accuracy: 0.5771
Epoch 5/100
24/24 [==============================] - 4s 147ms/step - loss: 0.5047 - accuracy: 0.7687 - val_loss: 0.6924 - val_accuracy: 0.5771
Epoch 6/100
24/24 [==============================] - 3s 138ms/step - loss: 0.4870 - accuracy: 0.7865 - val_loss: 0.6927 - val_accuracy: 0.5232
Epoch 7/100
24/24 [==============================] - 3s 142ms/step - loss: 0.4651 - accuracy: 0.8096 - val_loss: 0.6927 - val_accuracy: 0.5771

Epoch 11/100
24/24 [==============================] - 4s 157ms/step - loss: 0.4252 - accuracy: 0.8301 - val_loss: 0.6869 - val_accuracy: 0.5913
Epoch 12/100
24/24 [==============================] - 3s 141ms/step - loss: 0.4251 - accuracy: 0.8242 - val_loss: 0.5709 - val_accuracy: 0.7642
Epoch 13/100
24/24 [==============================] - 3s 142ms/step - loss: 0.4090 - accuracy: 0.8393 - val_loss: 0.4557 - val_accuracy: 0.8353
Epoch 14/100
24/24 [==============================] - 4s 169ms/step - loss: 0.4048 - accuracy: 0.8367 - val_loss: 0.6529 - val_accuracy: 0.6302
Epoch 15/100
24/24 [==============================] - 4s 158ms/step - loss: 0.3927 - accuracy: 0.8466 - val_loss: 0.6935 - val_accuracy: 0.4229
Epoch 16/100
24/24 [==============================] - 4s 162ms/step - loss: 0.3904 - accuracy: 0.8509 - val_loss: 0.6629 - val_accuracy: 0.6557
Epoch 17/100
24/24 [==============================] - 4s 173ms/step - loss: 0.3792 - accuracy: 0.8499 - val_loss: 0.4679 - val_accuracy:

24/24 [==============================] - 3s 146ms/step - loss: 0.3543 - accuracy: 0.8644 - val_loss: 0.3979 - val_accuracy: 0.8428
Epoch 22/100
24/24 [==============================] - 4s 155ms/step - loss: 0.3413 - accuracy: 0.8684 - val_loss: 0.5153 - val_accuracy: 0.7425
Epoch 23/100
24/24 [==============================] - 4s 159ms/step - loss: 0.3375 - accuracy: 0.8700 - val_loss: 0.7329 - val_accuracy: 0.4229
Epoch 24/100
24/24 [==============================] - 3s 142ms/step - loss: 0.3440 - accuracy: 0.8664 - val_loss: 0.3821 - val_accuracy: 0.8383
Epoch 25/100
24/24 [==============================] - 3s 140ms/step - loss: 0.3133 - accuracy: 0.8839 - val_loss: 0.3358 - val_accuracy: 0.8623
Epoch 26/100
24/24 [==============================] - 4s 152ms/step - loss: 0.3217 - accuracy: 0.8806 - val_loss: 0.4861 - val_accuracy: 0.7627
Epoch 27/100
24/24 [==============================] - 3s 141ms/step - loss: 0.3108 - accuracy: 0.8842 - val_loss: 0.3606 - val_accuracy: 0.8436
Epoch

24/24 [==============================] - 4s 153ms/step - loss: 0.2959 - accuracy: 0.8822 - val_loss: 1.4091 - val_accuracy: 0.4356
Epoch 33/100
24/24 [==============================] - 4s 146ms/step - loss: 0.3168 - accuracy: 0.8726 - val_loss: 0.4425 - val_accuracy: 0.7912
Epoch 34/100
24/24 [==============================] - 4s 162ms/step - loss: 0.3037 - accuracy: 0.8753 - val_loss: 1.3413 - val_accuracy: 0.4798
Epoch 35/100
24/24 [==============================] - 3s 145ms/step - loss: 0.2915 - accuracy: 0.8905 - val_loss: 0.5016 - val_accuracy: 0.7530
Epoch 36/100
24/24 [==============================] - 4s 153ms/step - loss: 0.2963 - accuracy: 0.8858 - val_loss: 1.3474 - val_accuracy: 0.4499
Epoch 37/100
24/24 [==============================] - 3s 145ms/step - loss: 0.2687 - accuracy: 0.8981 - val_loss: 0.3358 - val_accuracy: 0.8570
Epoch 38/100
24/24 [==============================] - 3s 143ms/step - loss: 0.2757 - accuracy: 0.8957 - val_loss: 0.6091 - val_accuracy: 0.5135
Epoch

24/24 [==============================] - 3s 145ms/step - loss: 0.2443 - accuracy: 0.8997 - val_loss: 0.3889 - val_accuracy: 0.8323
Epoch 44/100
24/24 [==============================] - 4s 148ms/step - loss: 0.2438 - accuracy: 0.9103 - val_loss: 0.3130 - val_accuracy: 0.8705
Epoch 45/100
24/24 [==============================] - 4s 151ms/step - loss: 0.2345 - accuracy: 0.9113 - val_loss: 0.3396 - val_accuracy: 0.8578
Epoch 46/100
24/24 [==============================] - 4s 147ms/step - loss: 0.2349 - accuracy: 0.9086 - val_loss: 0.3750 - val_accuracy: 0.8443
Epoch 47/100
24/24 [==============================] - 3s 137ms/step - loss: 0.2250 - accuracy: 0.9172 - val_loss: 0.3042 - val_accuracy: 0.8735
Epoch 48/100
24/24 [==============================] - 3s 142ms/step - loss: 0.2123 - accuracy: 0.9202 - val_loss: 0.3845 - val_accuracy: 0.8376
Epoch 49/100
24/24 [==============================] - 4s 160ms/step - loss: 0.2624 - accuracy: 0.8954 - val_loss: 0.7441 - val_accuracy: 0.6153
Epoch

24/24 [==============================] - 3s 142ms/step - loss: 0.2004 - accuracy: 0.9251 - val_loss: 0.3698 - val_accuracy: 0.8510
Epoch 55/100
24/24 [==============================] - 3s 143ms/step - loss: 0.1931 - accuracy: 0.9327 - val_loss: 0.3981 - val_accuracy: 0.8391
Epoch 56/100
24/24 [==============================] - 3s 142ms/step - loss: 0.1888 - accuracy: 0.9320 - val_loss: 0.2847 - val_accuracy: 0.8885
Epoch 57/100
24/24 [==============================] - 4s 150ms/step - loss: 0.1980 - accuracy: 0.9281 - val_loss: 0.3849 - val_accuracy: 0.8398
Epoch 58/100
24/24 [==============================] - 4s 148ms/step - loss: 0.1865 - accuracy: 0.9291 - val_loss: 0.3978 - val_accuracy: 0.8413
Epoch 59/100
24/24 [==============================] - 4s 153ms/step - loss: 0.1805 - accuracy: 0.9324 - val_loss: 0.3984 - val_accuracy: 0.8293
Epoch 60/100
24/24 [==============================] - 4s 163ms/step - loss: 0.1750 - accuracy: 0.9386 - val_loss: 0.3638 - val_accuracy: 0.8428
Epoch

24/24 [==============================] - 4s 160ms/step - loss: 0.1558 - accuracy: 0.9475 - val_loss: 0.4769 - val_accuracy: 0.7844
Epoch 65/100
24/24 [==============================] - 4s 152ms/step - loss: 0.1511 - accuracy: 0.9515 - val_loss: 0.4515 - val_accuracy: 0.8009
Epoch 66/100
24/24 [==============================] - 4s 150ms/step - loss: 0.1573 - accuracy: 0.9449 - val_loss: 0.3308 - val_accuracy: 0.8668
Epoch 67/100
24/24 [==============================] - 4s 164ms/step - loss: 0.1478 - accuracy: 0.9515 - val_loss: 0.3832 - val_accuracy: 0.8338
Epoch 68/100
24/24 [==============================] - 4s 153ms/step - loss: 0.1334 - accuracy: 0.9571 - val_loss: 0.5581 - val_accuracy: 0.7545
Epoch 69/100
24/24 [==============================] - 4s 148ms/step - loss: 0.1447 - accuracy: 0.9512 - val_loss: 0.4652 - val_accuracy: 0.8046
Epoch 70/100
24/24 [==============================] - 4s 146ms/step - loss: 0.1477 - accuracy: 0.9469 - val_loss: 0.2924 - val_accuracy: 0.8870
Epoch

24/24 [==============================] - 4s 163ms/step - loss: 0.1256 - accuracy: 0.9594 - val_loss: 0.5362 - val_accuracy: 0.7762
Epoch 76/100
24/24 [==============================] - 3s 145ms/step - loss: 0.1299 - accuracy: 0.9551 - val_loss: 0.5397 - val_accuracy: 0.8046
Epoch 77/100
24/24 [==============================] - 4s 159ms/step - loss: 0.1125 - accuracy: 0.9660 - val_loss: 0.3242 - val_accuracy: 0.8743
Epoch 78/100
24/24 [==============================] - 3s 145ms/step - loss: 0.1119 - accuracy: 0.9654 - val_loss: 0.3231 - val_accuracy: 0.8802
Epoch 79/100
24/24 [==============================] - 4s 159ms/step - loss: 0.1022 - accuracy: 0.9670 - val_loss: 0.3054 - val_accuracy: 0.8810
Epoch 80/100
24/24 [==============================] - 4s 179ms/step - loss: 0.0970 - accuracy: 0.9729 - val_loss: 0.4653 - val_accuracy: 0.8174
Epoch 81/100
24/24 [==============================] - 4s 169ms/step - loss: 0.0949 - accuracy: 0.9743 - val_loss: 0.5108 - val_accuracy: 0.8069
Epoch

24/24 [==============================] - 4s 162ms/step - loss: 0.0932 - accuracy: 0.9703 - val_loss: 0.3539 - val_accuracy: 0.8780
Epoch 86/100
24/24 [==============================] - 4s 155ms/step - loss: 0.0902 - accuracy: 0.9716 - val_loss: 0.3897 - val_accuracy: 0.8578
Epoch 87/100
24/24 [==============================] - 4s 166ms/step - loss: 0.0921 - accuracy: 0.9683 - val_loss: 0.4322 - val_accuracy: 0.8488
Epoch 88/100
24/24 [==============================] - 3s 141ms/step - loss: 0.0837 - accuracy: 0.9762 - val_loss: 0.3472 - val_accuracy: 0.8780
Epoch 89/100
24/24 [==============================] - 4s 157ms/step - loss: 0.0739 - accuracy: 0.9802 - val_loss: 0.3695 - val_accuracy: 0.8780
Epoch 90/100
24/24 [==============================] - 4s 156ms/step - loss: 0.1027 - accuracy: 0.9670 - val_loss: 0.5060 - val_accuracy: 0.8166
Epoch 91/100
24/24 [==============================] - 3s 142ms/step - loss: 0.0683 - accuracy: 0.9812 - val_loss: 0.3958 - val_accuracy: 0.8510
Epoch

24/24 [==============================] - 4s 150ms/step - loss: 0.0673 - accuracy: 0.9809 - val_loss: 0.6259 - val_accuracy: 0.7949
Epoch 97/100
24/24 [==============================] - 4s 171ms/step - loss: 0.0820 - accuracy: 0.9713 - val_loss: 0.3931 - val_accuracy: 0.8645
Epoch 98/100
24/24 [==============================] - 4s 161ms/step - loss: 0.0540 - accuracy: 0.9852 - val_loss: 0.5749 - val_accuracy: 0.8316
Epoch 99/100
24/24 [==============================] - 4s 155ms/step - loss: 0.0598 - accuracy: 0.9838 - val_loss: 0.5027 - val_accuracy: 0.8084
Epoch 100/100
24/24 [==============================] - 4s 153ms/step - loss: 0.0919 - accuracy: 0.9726 - val_loss: 0.3910 - val_accuracy: 0.8668


In [ ]:
## Saving the model


In [62]:
def evaluate_model(model, train_gen, val_gen):
    
    print(get_classification_report(model, train_gen, 'Training Data'))
    print(get_classification_report(model, val_gen, 'Validation Data'))

def get_classification_report(model, gen, res_string):
    
    X, y, pred = generate_all_data(generator=gen, 
                             predict=True,
                             model=model)
    
    print('\n################################################\n')
    print('Accuracy score for {}:\t{}'.format(res_string, accuracy_score(y, pred)))
    
    print('\n################################################\n')
    print('Classification report for {}:\t\n{}'.format(res_string, classification_report(y, pred)))
    
    print('\n################################################\n')

    
def generate_all_data(generator, predict=False, model=None):
    
    X, y, pred = [], [], []
    length = generator.__len__()
    
    for i in tqdm(range(length)):
        
        data, label = generator.__getitem__(i)
        X.append(data)
        y.append(np.argmax(label, axis=1))
        if predict:
            p = model.predict(data)
            pred.append(np.argmax(p, axis=1))
    
    X = np.concatenate(X)
    y = np.concatenate(y)
    if predict:
        pred = np.concatenate(pred)
        return X, y, pred
    
    return X, y

In [56]:
# Testing the generate_all_data function
X, y= generate_all_data(train_generator)

100%|██████████| 24/24 [00:02<00:00, 11.11it/s]


In [63]:
evaluate_model(model, train_generator, val_generator)

  9%|▉         | 1/11 [00:00<00:01,  6.95it/s]


################################################

Accuracy score for Training Data:	0.9353348729792148

################################################

Classification report for Training Data:	
              precision    recall  f1-score   support

           0       0.87      0.99      0.92      1209
           1       0.99      0.90      0.94      1822

    accuracy                           0.94      3031
   macro avg       0.93      0.94      0.93      3031
weighted avg       0.94      0.94      0.94      3031


################################################

None


100%|██████████| 11/11 [00:01<00:00,  7.57it/s]


################################################

Accuracy score for Validation Data:	0.8667664670658682

################################################

Classification report for Validation Data:	
              precision    recall  f1-score   support

           0       0.81      0.90      0.85       565
           1       0.92      0.84      0.88       771

    accuracy                           0.87      1336
   macro avg       0.86      0.87      0.87      1336
weighted avg       0.87      0.87      0.87      1336


################################################

None
